In [1]:
import re
from nltk.corpus.reader.indian import IndianCorpusView
dataset = IndianCorpusView('Official-Docs-Data.txt','utf-8', True, True,)
data=[]

with dataset as lines:
    for line in lines:
        data.append(line)


# data preprocessing

In [2]:

sentence=[]
tags=[]
temp_sentence=[]
temp_tag=[]
tag_finder="[A-Z]+" 
word_finder='[^A-Z]+'
for word in data:
  # print(word[0][0])
  if word==[]:#eliminating the empty list
    continue
  else:
    tag=re.findall(tag_finder,word[0][0])
    word=re.findall(word_finder,word[0][0])
    if tag[0]=="NUM": #condition for finding the end of the sentence
      if temp_sentence==['-'] or []:
        temp_sentence=[]
        temp_tag=[]
        continue
      sentence.append(temp_sentence)
      tags.append(temp_tag)
      temp_sentence=[]
      temp_tag=[]
    else:
      temp_sentence.append(word[0][0:len(word[0])-1])
      temp_tag.append(tag[0])
    # print(tag,word)

In [3]:
print(sentence)

[['ஆம்', 'பணிப்பாளர்கள்', 'அறிக்கை'], ['ஆம்', 'ஆண்டு', 'காலப்பிரிவில்', 'துறை', 'வாரியான', 'ஒப்பந்தங்கள்'], ['ஆம்', 'ஆண்டு', 'நாட்டின்', 'பங்கேற்பு', 'உபாயம்'], ['ஆம்', 'ஆண்டு', 'முன்மொழியப்பட்ட', 'மொத்தக்', 'கடன்கள்'], ['ஆம்', 'திகதி', 'அன்று', 'எமது', 'நிறுவனத்திற்கு', 'இந்த', 'வாகனம்', 'மாற்றப்பட்டது', '.'], ['ஆம்', 'திகதி', 'இரத்தினபுரியில்', 'நடைபெற்ற', 'தெசதிய', 'பாடசாலை', 'ஊடகச்', 'செயலமர்வு'], ['ஆம்', 'திகதி', 'எம்பிலிபிடிய', 'மொழி', 'அமைப்புக்கள்', 'மாநாடு', 'இடம்', 'பெற்றது', '.'], ['ஆம்', 'திகதி', 'கிளிநொச்சி', 'மொழி', 'அமைப்புக்கள்', 'செயற்பாட்டுகள்', 'குழு', 'தாபிக்கப்பட்டது', '.'], ['ஆம்', 'திகதிக்கு', 'செடிப்பண்ணை', 'மற்றும்', 'ஆய்வுகூடத்து', 'பணியாளர்கள்', 'விபரம்'], ['திகதிக்கு', 'முன்னர்', 'கீழ்வரும்', 'முகவரிக்குக்', 'கிடைப்பதற்கு', 'ஆவண', 'செய்யுங்கள்', '.'], ['ஆம்', 'திகதி', 'தனமல்வில', 'பிரதேச', 'செயலகத்தில்', 'மொழி', 'கருத்தரங்கொன்று', 'இடம்பெற்றது', '.'], ['திகதி', 'நடைபெற்ற', 'அமைச்சரவைக்', 'குழுக்', 'கூட்டத்தில்', 'தீர்மானம்', '.'], ['ஆம்', 'திகதி', 'நடைபெற்ற

In [4]:
print(tags)

[['CCONJ', 'NOUN', 'NOUN'], ['CCONJ', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'NOUN'], ['CCONJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN'], ['CCONJ', 'NOUN', 'VERB', 'NOUN', 'NOUN'], ['CCONJ', 'NOUN', 'ADV', 'PRON', 'NOUN', 'DET', 'NOUN', 'VERB', 'PUNCT'], ['CCONJ', 'NOUN', 'PROPN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN'], ['CCONJ', 'NOUN', 'PROPN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PUNCT'], ['CCONJ', 'NOUN', 'PROPN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PUNCT'], ['CCONJ', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'NOUN', 'NOUN'], ['NOUN', 'ADV', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'PUNCT'], ['CCONJ', 'NOUN', 'PROPN', 'NOUN', 'NOUN', 'NOUN', 'ADV', 'VERB', 'PUNCT'], ['NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PUNCT'], ['CCONJ', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN'], ['CCONJ', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN'], ['NOUN', 'VERB'], ['NOUN', 'ADP', 'NOUN', 'NOUN', 'VERB', 'NOUN'], ['NOUN', 'ADP', 'NOUN', 'NOUN', 'VERB', 'VERB', 'VAUX', 'PUNCT'], ['CCONJ', 'N

In [5]:
from nltk import flatten
temp_tags=flatten(tags)
unique_tags=[]
for tag in temp_tags:
    if tag in unique_tags:
        pass
    else:
        if tag != []:#eliminating the empty tags
            unique_tags.append(tag)
        else:
            pass
print(unique_tags)
print(f"there are {len(unique_tags)} unique tags")

['CCONJ', 'NOUN', 'ADJ', 'VERB', 'ADV', 'PRON', 'DET', 'PUNCT', 'PROPN', 'ADP', 'VAUX', 'UPS', 'H', 'ICTAD', 'NFC', 'SLAB', 'I', 'III', 'USD', 'SI', 'KMC', 'AGM', 'L', 'R', 'VII', 'UHF', 'SWEDAC']
there are 27 unique tags


In [6]:
index=[]
for x in unique_tags:
    for y in unique_tags:
        index.append((x,y))
print(len(index))

729


#### calculating transition probability

In [8]:

#find how likely a tag is followed by another
#denominator
from collections import defaultdict
tag_counter=defaultdict(int)
tag_counts={}
all_tags=flatten(tags)
for unique in unique_tags:
    for tag in all_tags:
        if unique==tag:
            tag_counter[unique]
            tag_counter[unique]+=1
tag_counter
    

defaultdict(int,
            {'CCONJ': 406,
             'NOUN': 5064,
             'ADJ': 334,
             'VERB': 910,
             'ADV': 211,
             'PRON': 152,
             'DET': 71,
             'PUNCT': 843,
             'PROPN': 314,
             'ADP': 102,
             'VAUX': 40,
             'UPS': 1,
             'H': 1,
             'ICTAD': 2,
             'NFC': 1,
             'SLAB': 1,
             'I': 1,
             'III': 1,
             'USD': 1,
             'SI': 1,
             'KMC': 1,
             'AGM': 1,
             'L': 1,
             'R': 1,
             'VII': 1,
             'UHF': 1,
             'SWEDAC': 1})

In [9]:
# numerator
i=0
while i<len(temp_tags):
    print(temp_tags[i])
    i+=1

CCONJ
NOUN
NOUN
CCONJ
NOUN
NOUN
NOUN
ADJ
NOUN
CCONJ
NOUN
NOUN
NOUN
NOUN
CCONJ
NOUN
VERB
NOUN
NOUN
CCONJ
NOUN
ADV
PRON
NOUN
DET
NOUN
VERB
PUNCT
CCONJ
NOUN
PROPN
VERB
NOUN
NOUN
NOUN
NOUN
CCONJ
NOUN
PROPN
NOUN
NOUN
NOUN
NOUN
VERB
PUNCT
CCONJ
NOUN
PROPN
NOUN
NOUN
NOUN
NOUN
VERB
PUNCT
CCONJ
NOUN
NOUN
CCONJ
NOUN
NOUN
NOUN
NOUN
ADV
NOUN
NOUN
NOUN
NOUN
VERB
PUNCT
CCONJ
NOUN
PROPN
NOUN
NOUN
NOUN
ADV
VERB
PUNCT
NOUN
VERB
NOUN
NOUN
NOUN
NOUN
PUNCT
CCONJ
NOUN
VERB
NOUN
NOUN
NOUN
NOUN
CCONJ
NOUN
NOUN
VERB
NOUN
NOUN
NOUN
NOUN
NOUN
NOUN
VERB
NOUN
ADP
NOUN
NOUN
VERB
NOUN
NOUN
ADP
NOUN
NOUN
VERB
VERB
VAUX
PUNCT
CCONJ
NOUN
NOUN
NOUN
NOUN
NOUN
NOUN
NOUN
NOUN
CCONJ
NOUN
NOUN
CCONJ
NOUN
PROPN
PROPN
NOUN
VERB
PROPN
NOUN
NOUN
NOUN
NOUN
VERB
NOUN
NOUN
VERB
PUNCT
NOUN
ADJ
NOUN
ADJ
NOUN
NOUN
NOUN
NOUN
ADJ
NOUN
VAUX
PUNCT
ADP
ADP
PROPN
PROPN
PROPN
NOUN
VERB
PUNCT
CCONJ
CCONJ
NOUN
NOUN
NOUN
PUNCT
ADP
ADP
PROPN
PROPN
NOUN
NOUN
VERB
PUNCT
NOUN
NOUN
VERB
NOUN
NOUN
NOUN
NOUN
NOUN
NOUN
NOUN
VERB
NOUN
PUNCT
CCONJ
NOUN


In [10]:
numerator=defaultdict(int)
for i in index:
    # print(i[0],i[1])
    j=0
    while j<len(temp_tags):
        try:
            if temp_tags[j]==i[0] and temp_tags[j+1]==i[1]:
                # print(temp_tags[j],temp_tags[j+1])
                numerator[i]+=1
            j+=1
        except IndexError:
            j+=1
            pass

In [11]:
transition_table=defaultdict(float)
for cell in index:
    # print(cell)
    # print(numerator[cell])
    # print(tag_counter[cell[0]])
    transition_table[cell]=numerator[cell]/tag_counter[cell[0]] 

In [12]:
transition_table

defaultdict(float,
            {('CCONJ', 'CCONJ'): 0.007389162561576354,
             ('CCONJ', 'NOUN'): 0.7783251231527094,
             ('CCONJ', 'ADJ'): 0.0665024630541872,
             ('CCONJ', 'VERB'): 0.04433497536945813,
             ('CCONJ', 'ADV'): 0.009852216748768473,
             ('CCONJ', 'PRON'): 0.022167487684729065,
             ('CCONJ', 'DET'): 0.0049261083743842365,
             ('CCONJ', 'PUNCT'): 0.022167487684729065,
             ('CCONJ', 'PROPN'): 0.03940886699507389,
             ('CCONJ', 'ADP'): 0.0024630541871921183,
             ('CCONJ', 'VAUX'): 0.0,
             ('CCONJ', 'UPS'): 0.0024630541871921183,
             ('CCONJ', 'H'): 0.0,
             ('CCONJ', 'ICTAD'): 0.0,
             ('CCONJ', 'NFC'): 0.0,
             ('CCONJ', 'SLAB'): 0.0,
             ('CCONJ', 'I'): 0.0,
             ('CCONJ', 'III'): 0.0,
             ('CCONJ', 'USD'): 0.0,
             ('CCONJ', 'SI'): 0.0,
             ('CCONJ', 'KMC'): 0.0,
             ('CCONJ', 'AGM'): 0.

#### calculating emission probability

In [13]:

words=[]
temp_word=[]
tag_finder="[A-Z]+" 
word_finder='[^A-Z]+'
for word in data:
  # print(word[0][0])
  if word==[]:#eliminating the empty list
    continue
  else:
    tag=re.findall(tag_finder,word[0][0])
    word=re.findall(word_finder,word[0][0])
    if tag[0]=="NUM": #condition for finding the end of the sentence
      if temp_sentence==['-'] or []:
        temp_word=[]
        continue
      words.append(temp_word)
      temp_word=[]
    else:
      temp_word.append((word[0][0:len(word[0])-1],tag[0]))

    # print(tag,word)

In [14]:
words

[[('ஆம்', 'CCONJ'), ('பணிப்பாளர்கள்', 'NOUN'), ('அறிக்கை', 'NOUN')],
 [('-', 'NOUN')],
 [('ஆம்', 'CCONJ'),
  ('ஆண்டு', 'NOUN'),
  ('காலப்பிரிவில்', 'NOUN'),
  ('துறை', 'NOUN'),
  ('வாரியான', 'ADJ'),
  ('ஒப்பந்தங்கள்', 'NOUN')],
 [('-', 'NOUN')],
 [('ஆம்', 'CCONJ'),
  ('ஆண்டு', 'NOUN'),
  ('நாட்டின்', 'NOUN'),
  ('பங்கேற்பு', 'NOUN'),
  ('உபாயம்', 'NOUN')],
 [('-', 'NOUN')],
 [('ஆம்', 'CCONJ'),
  ('ஆண்டு', 'NOUN'),
  ('முன்மொழியப்பட்ட', 'VERB'),
  ('மொத்தக்', 'NOUN'),
  ('கடன்கள்', 'NOUN')],
 [('ஆம்', 'CCONJ'),
  ('திகதி', 'NOUN'),
  ('அன்று', 'ADV'),
  ('எமது', 'PRON'),
  ('நிறுவனத்திற்கு', 'NOUN'),
  ('இந்த', 'DET'),
  ('வாகனம்', 'NOUN'),
  ('மாற்றப்பட்டது', 'VERB'),
  ('.', 'PUNCT')],
 [('ஆம்', 'CCONJ'),
  ('திகதி', 'NOUN'),
  ('இரத்தினபுரியில்', 'PROPN'),
  ('நடைபெற்ற', 'VERB'),
  ('தெசதிய', 'NOUN'),
  ('பாடசாலை', 'NOUN'),
  ('ஊடகச்', 'NOUN'),
  ('செயலமர்வு', 'NOUN')],
 [('ஆம்', 'CCONJ'),
  ('திகதி', 'NOUN'),
  ('எம்பிலிபிடிய', 'PROPN'),
  ('மொழி', 'NOUN'),
  ('அமைப்புக்கள்', 'NOUN

In [15]:
comb=defaultdict(int)
for word in words:
    ele=0
    while ele<len(word):
        # print(word[ele])
        comb[word[ele]]+=1
        ele+=1

In [16]:
comb
#('அறிக்கை', 'NOUN'): 9 says that 'அறிக்கை' is classified as noun 9 times

defaultdict(int,
            {('ஆம்', 'CCONJ'): 106,
             ('பணிப்பாளர்கள்', 'NOUN'): 2,
             ('அறிக்கை', 'NOUN'): 9,
             ('-', 'NOUN'): 24,
             ('ஆண்டு', 'NOUN'): 17,
             ('காலப்பிரிவில்', 'NOUN'): 2,
             ('துறை', 'NOUN'): 6,
             ('வாரியான', 'ADJ'): 1,
             ('ஒப்பந்தங்கள்', 'NOUN'): 2,
             ('நாட்டின்', 'NOUN'): 4,
             ('பங்கேற்பு', 'NOUN'): 1,
             ('உபாயம்', 'NOUN'): 1,
             ('முன்மொழியப்பட்ட', 'VERB'): 3,
             ('மொத்தக்', 'NOUN'): 1,
             ('கடன்கள்', 'NOUN'): 2,
             ('திகதி', 'NOUN'): 19,
             ('அன்று', 'ADV'): 1,
             ('எமது', 'PRON'): 2,
             ('நிறுவனத்திற்கு', 'NOUN'): 2,
             ('இந்த', 'DET'): 36,
             ('வாகனம்', 'NOUN'): 3,
             ('மாற்றப்பட்டது', 'VERB'): 1,
             ('.', 'PUNCT'): 679,
             ('இரத்தினபுரியில்', 'PROPN'): 1,
             ('நடைபெற்ற', 'VERB'): 5,
             ('தெசதிய', 'NOUN'):

In [17]:
emission_prob=defaultdict(float)
for key,values in comb.items():
    emission_prob[key]=values/tag_counter[key[1]]

In [18]:
emission_prob

defaultdict(float,
            {('ஆம்', 'CCONJ'): 0.26108374384236455,
             ('பணிப்பாளர்கள்', 'NOUN'): 0.00039494470774091627,
             ('அறிக்கை', 'NOUN'): 0.0017772511848341231,
             ('-', 'NOUN'): 0.004739336492890996,
             ('ஆண்டு', 'NOUN'): 0.0033570300157977884,
             ('காலப்பிரிவில்', 'NOUN'): 0.00039494470774091627,
             ('துறை', 'NOUN'): 0.001184834123222749,
             ('வாரியான', 'ADJ'): 0.0029940119760479044,
             ('ஒப்பந்தங்கள்', 'NOUN'): 0.00039494470774091627,
             ('நாட்டின்', 'NOUN'): 0.0007898894154818325,
             ('பங்கேற்பு', 'NOUN'): 0.00019747235387045813,
             ('உபாயம்', 'NOUN'): 0.00019747235387045813,
             ('முன்மொழியப்பட்ட', 'VERB'): 0.0032967032967032967,
             ('மொத்தக்', 'NOUN'): 0.00019747235387045813,
             ('கடன்கள்', 'NOUN'): 0.00039494470774091627,
             ('திகதி', 'NOUN'): 0.0037519747235387046,
             ('அன்று', 'ADV'): 0.004739336492890996,
  

checking how many words has more than one tag in the dataset

In [19]:
unique_words=[]
for word in words:
    ele=0
    while ele<len(word):
        if word[ele][0] not in unique_words:
            unique_words.append(word[ele][0])
        ele+=1

In [20]:

word_tag=[]
temp_sentence=[]
tag_finder="[A-Z]+" 
word_finder='[^A-Z]+'
for word in data:
  # print(word[0][0])
  if word==[]:#eliminating the empty list
    continue
  else:
    tag=re.findall(tag_finder,word[0][0])
    word=re.findall(word_finder,word[0][0])
    if tag[0]=="NUM": #condition for finding the end of the sentence
      if temp_sentence==['-'] or []:
        continue
    else:
      word_tag.append((word[0][0:len(word[0])-1],tag[0]))
    # print(tag,word)

In [21]:
word_tag

[('ஆம்', 'CCONJ'),
 ('பணிப்பாளர்கள்', 'NOUN'),
 ('அறிக்கை', 'NOUN'),
 ('-', 'NOUN'),
 ('ஆம்', 'CCONJ'),
 ('ஆண்டு', 'NOUN'),
 ('காலப்பிரிவில்', 'NOUN'),
 ('துறை', 'NOUN'),
 ('வாரியான', 'ADJ'),
 ('ஒப்பந்தங்கள்', 'NOUN'),
 ('-', 'NOUN'),
 ('ஆம்', 'CCONJ'),
 ('ஆண்டு', 'NOUN'),
 ('நாட்டின்', 'NOUN'),
 ('பங்கேற்பு', 'NOUN'),
 ('உபாயம்', 'NOUN'),
 ('-', 'NOUN'),
 ('ஆம்', 'CCONJ'),
 ('ஆண்டு', 'NOUN'),
 ('முன்மொழியப்பட்ட', 'VERB'),
 ('மொத்தக்', 'NOUN'),
 ('கடன்கள்', 'NOUN'),
 ('ஆம்', 'CCONJ'),
 ('திகதி', 'NOUN'),
 ('அன்று', 'ADV'),
 ('எமது', 'PRON'),
 ('நிறுவனத்திற்கு', 'NOUN'),
 ('இந்த', 'DET'),
 ('வாகனம்', 'NOUN'),
 ('மாற்றப்பட்டது', 'VERB'),
 ('.', 'PUNCT'),
 ('ஆம்', 'CCONJ'),
 ('திகதி', 'NOUN'),
 ('இரத்தினபுரியில்', 'PROPN'),
 ('நடைபெற்ற', 'VERB'),
 ('தெசதிய', 'NOUN'),
 ('பாடசாலை', 'NOUN'),
 ('ஊடகச்', 'NOUN'),
 ('செயலமர்வு', 'NOUN'),
 ('ஆம்', 'CCONJ'),
 ('திகதி', 'NOUN'),
 ('எம்பிலிபிடிய', 'PROPN'),
 ('மொழி', 'NOUN'),
 ('அமைப்புக்கள்', 'NOUN'),
 ('மாநாடு', 'NOUN'),
 ('இடம்', 'NOUN'),
 ('பெ

In [22]:
temp1=defaultdict(list)
for word in unique_words:
    # print(word)
    for comb in word_tag:
        if comb[0]==word:
            if comb[1] not in temp1[word]:
                temp1[word].append(comb[1])

In [23]:
# word which has more than one tag
for key,values in temp1.items():
    if len(values)>1:
        print(key,values)

- ['NOUN', 'PROPN', 'AGM']
கிளிநொச்சி ['PROPN', 'NOUN']
சிரேஷ்ட ['ADJ', 'NOUN']
ரணவிரு ['NOUN', 'PROPN']
இல் ['ADV', 'PROPN', 'VERB']
சபையானது ['NOUN', 'VERB']
 ['UPS', 'ICTAD', 'NFC', 'SLAB', 'I', 'SI', 'L', 'UHF', 'SWEDAC']
பழுதுபார்த்தல் ['NOUN', 'VERB']
றணவிரு ['PROPN', 'NOUN']
பிள்ளையார் ['PROPN', 'NOUN']
தாபித்தல் ['VERB', 'PROPN']
ஒருங்கிணைத்தல் ['VERB', 'NOUN']
விருத்தி ['NOUN', 'VERB']
/ ['NOUN', 'VERB']
நிர்மாணித்தல் ['NOUN', 'PROPN']
முன்னைய ['ADJ', 'NOUN']
ஜனாதிபதி ['NOUN', 'PROPN']
லங்கா ['NOUN', 'PROPN']
பின்வரும் ['NOUN', 'ADJ', 'VERB']
இரத்தினபுரி ['PROPN', 'NOUN']
இந்துப் ['NOUN', 'PROPN']
இலங்கைக் ['PROPN', 'NOUN']
விழிப்பூட்டல் ['NOUN', 'PROPN']
முப்பரிமாண ['ADJ', 'NOUN']
இந் ['NOUN', 'ADJ']
கொழும்பு ['PROPN', 'NOUN']
மேற்கொள்ளல் ['NOUN', 'VERB']
கை.தொ.நி. ['NOUN', 'PROPN']
எம்மால் ['VERB', 'PRON']
: ['NOUN', 'PROPN']
பங்குபற்றச் ['NOUN', 'VERB']
தயவுடன் ['PRON', 'NOUN']
மேம்படுத்தல் ['NOUN', 'VERB']
தெரிவு ['NOUN', 'VERB']
சமர்ப்பித்தல் ['VERB', 'NOUN']
வைத்தியப் ['

# Tagging

In [24]:
#tokenize every word 
#find all possible tags for each word
#buid different combinations
#calculate prob

In [240]:
import copy
temp=defaultdict(list)
def seq_gen(tagged):
    only_tag=[]
    for key,values in tagged.items(): # makes only one run
        only_tag.append(values)
    cp_only_tag = copy.deepcopy(only_tag)
    #find lengthy tag
    length=0
    for i in only_tag:
        if len(i)>length:
            length=len(i)
    for i in only_tag:
        if len(i)<length:
            n=length-len(i)
            for j in range(n):
                i.append(0)
    initial_combo=[]
    for i in range(length):
        for j in only_tag:
            initial_combo.append(j[i])
    initial_combo=flatten(initial_combo)
    initial_combo=np.array(initial_combo).reshape(length,len(tagged))
    for combo in initial_combo:
        if '0' in combo:
            # print(combo)
            for tag in range(len(combo)):
                if combo[tag]=='0':
                    possible_tags=cp_only_tag[tag]
                    # print("missing tag position",tag,"possible tags are",cp_only_tag[tag])
                    if len(possible_tags)==1: #if the possible tag is only one 
                        combo[tag]=cp_only_tag[tag][0]
                    else:
                        pass
    # print(initial_combo)
    return initial_combo

def prob_calc(sequence,ip):
    # print(sequence,ip)
    transition=[]
    for i in sequence:
        trans=[]
        for j in range(len(i)):
            try:
                trans.append((i[j],i[j+1]))
            except IndexError:
                pass
        transition.append(trans)
    for i in range(len(transition)):
        for j in range(len(transition[i])):
            # print(i,j)
            # print(transition[i][j])
            # print(transition_table[transition[i][j]])
            transition[i][j]=transition_table[transition[i][j]]
    
    emis=[]
    for i in sequence:
        t=[]
        new=zip(ip,i)
        for j in new:
            # print(j)
            t.append(j)
        emis.append(t)
    for i in range(len(emis)):
        for j in range(len(emis[i])):
            # print(i,j)
            # print(emis[i][j])
            # print(emission_prob[emis[i][j]])
            emis[i][j]=emission_prob[emis[i][j]]
    return emis,transition

def best_finder(transition,emision):
    final=[]
    for i in range(len(transition)):
        # print(transition[i])
        transition[i]=np.prod(transition[i])
    for i in range(len(emision)):
        # print(emision[i])
        emision[i]=np.prod(emision[i])
    for i in range(len(transition)):
        final.append(transition[i]*emision[i])
    return final.index(max(final))    
def tagger(ip):
    ip=ip.split(" ")
    ###for finding different tags that a word can take
    for word in ip:
        # print(word)
        for i in words:
            ele=0
            while ele<len(i):
                # print(i[ele])
                if word==i[ele][0]: # finding word match
                    if i[ele][1] not in temp[word]:
                        temp[word].append(i[ele][1])
                ele+=1
    # print("\n The possible tags that can be taken by the words in the sentence :\n ",temp)
    sequence=seq_gen(temp)
    # print("\n The possible tags that the given sentence can have is \n",sequence)
    # print(type(sequence))
    # print(sequence.shape)
    # print(sequence,ip)
    # print(type(sequence),type(ip))
    emision,transition=prob_calc(sequence,ip)
    best=best_finder(transition,emision)
    # print("best",best)
    print(ip)
    print(sequence[best])
    
tagger("ஜனவரி டிசம்பர் மாதம் வரையில் பின்வரும் கூட்டங்கள் நடாத்தப்பட்டன")


['ஜனவரி', 'டிசம்பர்', 'மாதம்', 'வரையில்', 'பின்வரும்', 'கூட்டங்கள்', 'நடாத்தப்பட்டன']
['PROPN' 'PROPN' 'NOUN' 'ADP' 'ADJ' 'NOUN' 'VERB']
